## Data Access demo

[OWSLib](https://geopython.github.io/OWSLib) is a Python package for client programming with Open Geospatial Consortium (OGC) web service (hence OWS) interface standards, and their related content models. In this demo we’ll work with the CSW, WMS and WCS interfaces.

In [ ]:
from owslib.csw import CatalogueServiceWeb
from owslib.wms import WebMapService
from owslib.wcs import WebCoverageService
import lxml.etree
import requests

### Data Discovery

The user has already discovered the dataset to use by using the OWSLib CSW client

In [ ]:
system_endpoint = 'https://resource-catalogue.185.52.193.87.nip.io/'

In [ ]:
csw = CatalogueServiceWeb(system_endpoint,timeout=30)

In [ ]:
#scene_id='S2B_MSIL2A_20200902T090559_N0214_R050_T34SGH_20200902T113910.SAFE'
scene_id='S2B_MSIL1C_20210402T095029_N0300_R079_T33SVB_20210402T121737.SAFE'

In [ ]:
csw.getrecordbyid(id=[scene_id])

In [ ]:
links = csw.records[scene_id].references

In [ ]:
for link in links:
    scheme = link['scheme']
    if 'WMS' in scheme:
        wms_endpoint=link['url']
        print(link['url'])

### Data Visualization

After discovering the dataset, the user can identify the WMS link and use the OWSLib WMs client to visualize the dataset

In [ ]:
wms = WebMapService(wms_endpoint, version='1.3.0')

The list of layers available to the WMS service:

In [ ]:
list(wms.contents)

Along with some WMS layer metadata:

In [ ]:
wms[scene_id].title

In [ ]:
wms[scene_id].boundingBoxWGS84

In [ ]:
[op.name for op in wms.operations]

In [ ]:
wms[scene_id].styles

In [ ]:
wms.getOperationByName('GetMap').formatOptions

The user can visualize the WMS GetMap request from matplotlib

In [ ]:
%matplotlib inline
import os, sys
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

def getMap(wms,layerName,bbox,filename):
    wms.getOperationByName('GetMap').formatOptions
    img = wms.getmap(layers=[layerName],
                 size=(600,300),
                 srs='EPSG:4326',
                 bbox=bbox,
                 format='image/png',
                 transparent=True)

    tmpfile = open(filename,'wb')
    tmpfile.write(img.read())
    tmpfile.close()

In [ ]:
getMap(wms,'S2B_MSIL1C_20210402T095029_N0300_R079_T33SVB_20210402T121737.SAFE__TRUE_COLOR',(13.8,36.9,15.1,37.9), 'rgb.png')
image1=mpimg.imread('rgb.png')
fig = plt.figure(figsize=(12,7))
img1=plt.imshow(image1,extent=[13.8,15.1,36.9,37.9],aspect='auto')
plt.show()

In [ ]:
getMap(wms,'S2B_MSIL2A_20200902T090559_N0214_R050_T34SGH_20200902T113910.SAFE__TRUE_COLOR',(23.4,37.8,24.5,38.8), 'rgb.png')
image1=mpimg.imread('rgb.png')
fig = plt.figure(figsize=(12,7))
img1=plt.imshow(image1,extent=[23.4,24.5,37.8,38.8],aspect='auto')
plt.show()

In [ ]:
wms2 = WebMapService('http://apps.ecmwf.int/wms/?token=public', version='1.3.0')
getMap(wms2,'foreground',(-180,-90,180,90), 'foreground.png')
getMap(wms2,'background',(-180,-90,180,90), 'background.png')
getMap(wms2,'composition_bbaod550',(-180,-90,180,90), 'bbaod550.png')
image1=mpimg.imread('background.png')
image2=mpimg.imread('bbaod550.png')
image3=mpimg.imread('foreground.png')
fig = plt.figure(figsize=(12,7))

img1=plt.imshow(image1,extent=[-180,180,-90,90],aspect='auto')
img2=plt.imshow(image2,extent=[-180,180,-90,90],aspect='auto')
img3=plt.imshow(image3,extent=[-180,180,-90,90],aspect='auto')
plt.show()

Alternatively, the Folium library is available to create a Leaflet map with the WMS layer

In [ ]:
import os
import folium

print(folium.__version__)

In [ ]:
m = folium.Map(location=[37, 14], zoom_start=6, tiles=None)


# folium.raster_layers.TileLayer(
#     tiles='http://{s}.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
#     attr='google',
#     name='google maps',
#     max_zoom=20,
#     subdomains=['mt0', 'mt1', 'mt2', 'mt3'],
#     overlay=False,
#     control=True,
# ).add_to(m)

folium.raster_layers.WmsTileLayer(
    url=wms_endpoint,
    layers='S2B_MSIL2A_20200902T090559_N0214_R050_T34SGH_20200902T113910.SAFE__TRUE_COLOR',
    name='S2B_MSIL2A_20200902T090559_N0214_R050_T34SGH_20200902T113910.SAFE__TRUE_COLOR',
    fmt='image/png',
    overlay=False,
    control=True,
).add_to(m)

folium.LayerControl().add_to(m)

#m.save(os.path.join('results', 'WMS_and_WMTS.html'))

m

### Data Download

After discovering and visualizing the dataset, the user can identify the WCS link from the catalogue record and use the OWSLib WCS client to download the dataset

In [ ]:
tree = None
for link in links:
    scheme = link['scheme']
    if 'WCS' in scheme:
        wcs_endpoint=link['url'].split('?')[0]
        wcs_id=link['url'].split('eoid=')[1]
        tree = lxml.etree.fromstring(requests.get(link['url']).content)
        break


coverage_ids = tree.xpath('wcs:CoverageDescriptions/wcs:CoverageDescription/@gml:id', namespaces=tree.nsmap)
coverage_ids

In [ ]:
from osgeo import gdal
gdal.UseExceptions()
getcoverage_request = wcs_endpoint + '?service=WCS&version=2.0.1&request=GetCoverage&coverageid=' + coverage_ids[1] + '&scaleSize=x(100),y(100)'
print(getcoverage_request)
ds = gdal.Open('/vsicurl/' + getcoverage_request)
data = ds.ReadAsArray()
data

In [ ]:
wcs = WebCoverageService(wcs_endpoint, version='2.0.1')

In [ ]:
wcs.contents.keys()